In [1]:
from src import environments
import nest_asyncio
nest_asyncio.apply()

import tensorflow as tf
from gym_derk.envs import DerkEnv
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from src.environments import EnvPerso, EnvTensorOne
import tf_agents
import reverb
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.specs import tensor_spec
from tf_agents.policies import py_tf_eager_policy
import reverb
from tf_agents.drivers import py_driver

from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
import tensorflow as tf
from tf_agents.agents.ppo import ppo_agent
from tf_agents.networks import actor_distribution_network, value_network

2023-07-04 22:56:46.867556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
env = EnvTensorOne()

In [3]:
# Define the network for the agent.
actor_net = actor_distribution_network.ActorDistributionNetwork(
    env.observation_tensor_spec(),
    env.action_tensor_spec())

value_net = value_network.ValueNetwork(
    env.observation_tensor_spec())

# Define the keras optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define the train step counter.
train_step_counter = tf.Variable(0)

# Create the PPO agent.
agent = ppo_agent.PPOAgent(
    env.time_step_spec(),
    env.action_tensor_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    train_step_counter=train_step_counter)

# Initialize the agent.
agent.initialize()

In [4]:
batch_size = 32
table_name = 'uniform_table'
replay_buffer_max_length = 100

In [17]:
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=10,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=9)

[reverb/cc/platform/tfrecord_checkpointer.cc:162]  Initializing TFRecordCheckpointer in /tmp/tmpruhneqoi.
[reverb/cc/platform/tfrecord_checkpointer.cc:567] Loading latest checkpoint from /tmp/tmpruhneqoi
[reverb/cc/platform/default/server.cc:71] Started replay server on port 41009


In [18]:
random_policy = random_tf_policy.RandomTFPolicy(env.time_step_spec(),
                                                env.action_spec())

In [19]:
initial_collect_steps = batch_size*2

In [20]:
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(env.reset())

(TimeStep(
 {'discount': <tf.Tensor: shape=(), dtype=float32, numpy=1.0>,
  'observation': <tf.Tensor: shape=(64,), dtype=float32, numpy=
 array([ 0.8833435 ,  1.        ,  0.56486577, -0.3423863 ,  0.12243512,
        -0.13617708,  0.1628796 , -0.05802261,  0.680625  ,  0.5482161 ,
         0.26040587,  0.33032417,  0.12312703, -0.04909642,  0.09416606,
         0.3091763 ,  1.        ,  0.02418862, -0.96681964,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.03493854, -0.0194951 ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,

In [10]:
env.close()

In [11]:
reward= {
    "damageEnemyStatue": 4,
    "damageEnemyUnit": 2,
    "killEnemyStatue": 4,
    "killEnemyUnit": 2,
    "healFriendlyStatue": 1,
    "healTeammatel": 2,
    "healTeammate2": 2,
    "timeSpentHomeBase": 0,
    "timeSpentHomeTerritory": 0,
    "timeSpentAwayTerritory": 0,
    "timeSpentAwayBase": 0,
    "damageTaken": -1,
    "friendlyFire": -1,
    "healEnemy": -1,
    "fallDamageTaken": -10,
    "statueDamageTaken": 0,
    "manualBonus": 0,
    "victory": 100,
    "loss": -100,
    "tie": 0,
    "teamSpirit": 0.5,
    "timeScaling": 1,
}

try:
    env.close()
finally:
    env = EnvTensorOne(reward_function = reward)

In [21]:
iterations = 1000
batch_size = 32
learning_rate = 0.001
nb_episodes = 2

In [22]:
agent.collect_data_spec

_TupleWrapper(Trajectory(
{'action': BoundedTensorSpec(shape=(15,), dtype=tf.float32, name='action', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': BoundedTensorSpec(shape=(64,), dtype=tf.float32, name='observation', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)),
 'policy_info': DictWrapper({'dist_params': DictWrapper({'loc': TensorSpec(shape=(15,), dtype=tf.float32, name='NormalProjectionNetwork_loc'), 'scale': TensorSpec(shape=(15,), dtype=tf.float32, name='NormalProjectionNetwork_scale')})}),
 'reward': BoundedTensorSpec(shape=(), dtype=tf.float32, name='reward', minimum=array(0., dtype=float32), maximum=array(inf, dtype=float32)),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')}))

In [24]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,  # Adjust this according to your machine's configuration.
    sample_batch_size=batch_size,
    num_steps=10).prefetch(3)

In [25]:
iterator = iter(dataset)

In [26]:
batch = next(iterator)

[reverb/cc/client.cc:165] Sampler and server are owned by the same process (1718) so Table uniform_table is accessed directly without gRPC.


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_13_device_/job:localhost/replica:0/task:0/device:CPU:0}} Inconsistent number of tensors received from table 'uniform_table'.  Specification has 8 tensors, but data coming from the table shows 6 tensors.
Table signature: 0: Tensor<name: 'step_type/step_type', dtype: int32, shape: [?]>, 1: Tensor<name: 'observation/observation', dtype: float, shape: [?,64]>, 2: Tensor<name: 'action/action', dtype: float, shape: [?,15]>, 3: Tensor<name: 'policy_info/dist_params/loc/NormalProjectionNetwork_loc', dtype: float, shape: [?,15]>, 4: Tensor<name: 'policy_info/dist_params/scale/NormalProjectionNetwork_scale', dtype: float, shape: [?,15]>, 5: Tensor<name: 'next_step_type/step_type', dtype: int32, shape: [?]>, 6: Tensor<name: 'reward/reward', dtype: float, shape: [?]>, 7: Tensor<name: 'discount/discount', dtype: float, shape: [?]>.
Incoming tensor signature: 0: Tensor<name: '', dtype: int32, shape: [9]>, 1: Tensor<name: '

In [40]:
def compute_avg_return(env, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    obs = env.reset()
    episode_return = 0.0
    done = False
    
    while not done:
      action_step = policy.action(obs)
      obs, reward, done, info = env.step(action_step.action)
      episode_return += reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  print(avg_return)
  return avg_return

In [60]:
env.close()

In [19]:
num_iterations = 1

In [42]:
#import py drive
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_step_driver

for _ in range(num_iterations):
    # Collect a few steps using collect_policy and save to the replay buffer.
    for _ in range(2):
        collect_driver.run()

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience)

    step = agent.train_step_counter.numpy()

    if step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss.loss))

(64,)
tf.Tensor(
[ 1.0000000e+00  1.0000000e+00  2.6377305e-01 -8.2191807e-01
  2.0597324e-01 -2.6229545e-01  2.7149928e-01 -3.8035849e-01
  1.0000000e+00 -4.3772139e-02  8.4088969e-01 -9.5503114e-02
  6.8007267e-01 -8.4140643e-02  6.8672717e-01 -8.0744218e-04
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  1.0000000e+00  1.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
 -4.6051260e-02 -1.2957668e-01  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00], shape

AttributeError: 'tuple' object has no attribute 'rank'

In [ ]:
# # Reset the train step.
# agent.train_step_counter.assign(0)

# # Evaluate the agent's policy once before training.
# avg_return = compute_avg_return(env, agent.policy, 2)
# returns = [avg_return]

# # Reset the environment.
# time_step = env.reset()

# # Create a driver to collect experience.
# collect_driver = dynamic_step_driver.DynamicStepDriver(
#     env,
#     agent.collect_policy,
#     observers=None,
#     num_steps=32) 

In [ ]:
# class Agent:
#     def __init__(self, shape_of_model: tuple):
#         nb_outputs = 15
#         self.model = keras.models.Sequential()
#         self.model.add(layers.Input(shape=(64,)))
#         for n in shape_of_model:
#             self.model.add(layers.Dense(units=n, activation='relu'))
#         self.model.add(layers.Dense(units=nb_outputs, activation='linear'))
    
#     def action(self, state: np.ndarray):
#         prediction = self.model.predict(state)
#         # min max scaling
        
#         return self.model.predict(state)
    


In [ ]:
# class AgentManager:
#     def __init__(self, env, number_of_agents):
#         self.env = env
#         self.agents = [Agent(shape_of_model=(15,15)) for i in range(number_of_agents)]
            
#     def take_action(self, observation):
#         return [agent.action(observation) for agent in self.agents]   
        
        